In [1]:
!pip install datasets
!pip install transformers datasets
!pip install streamlit
!pip install streamlit pyngrok
!npm install -g localtunnel
!pip install transformers

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, pipeline
import subprocess
import threading
import streamlit as st

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 28.0 MB/s 

# Step 1: Fine-Tune the Existing LLM on the Empathy Dataset

# Download and Prepare the Dataset:

In [2]:

# Load the dataset
dataset = load_dataset("ParlAI/blended_skill_talk", split="train[:1%]")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/980 [00:00<?, ? examples/s]

# Load the Pre-trained Model and Tokenizer:

In [3]:

model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# Tokenize the Dataset:

In [4]:
def tokenize_function(examples):
    inputs = tokenizer(examples['context'], padding="max_length", truncation=True)
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/48 [00:00<?, ? examples/s]

# Fine-Tune the Model:

In [5]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

# Save the fine-tuned model
model.save_pretrained("fine-tuned-empathy-model")
tokenizer.save_pretrained("fine-tuned-empathy-model")


Step,Training Loss


('fine-tuned-empathy-model/tokenizer_config.json',
 'fine-tuned-empathy-model/special_tokens_map.json',
 'fine-tuned-empathy-model/vocab.json',
 'fine-tuned-empathy-model/merges.txt',
 'fine-tuned-empathy-model/added_tokens.json',
 'fine-tuned-empathy-model/tokenizer.json')

# Step 2: Add a Prompt Engineering Layer
Create a Text Generation Pipeline:

In [6]:
# Load the fine-tuned model and tokenizer
model_name = "fine-tuned-empathy-model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


# Implement Prompt Engineering:

In [7]:
def generate_response(input_text):
    prompt = "Consider the customer’s previous interactions—how can we provide a response that shows understanding and care? "
    full_input = prompt + input_text
    response = generator(full_input, max_length=100, truncation=True, num_return_sequences=1)
    return response[0]['generated_text']

# Example
input_text = "I was really disappointed with the last product I received."
response = generate_response(input_text)
print(response)

Consider the customer’s previous interactions—how can we provide a response that shows understanding and care? I was really disappointed with the last product I received.


# Convert the Model into a Front-End or Chatbot Interface

In [11]:
!wget -q -O - ipv4.icanhazip.com


34.121.215.222


In [15]:
# Step 3: Convert the Model into a Front-End or Chatbot Interface

# Save the Streamlit app code into a file
%%writefile app.py

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import streamlit as st

# Load the fine-tuned model and tokenizer
model_name = "fine-tuned-empathy-model"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

st.title("Empathetic Chatbot")

user_input = st.text_input("Enter your message:")

if user_input:
    prompt = "Consider the customer’s previous interactions—how can we provide a response that shows understanding and care? "
    full_input = prompt + user_input
    response = generator(full_input, max_length=100, num_return_sequences=1)
    st.write("Chatbot Response:", response[0]['generated_text'])


Overwriting app.py


In [ ]:

# Function to run Streamlit app
def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py"])

# Start Streamlit in a separate thread
threading.Thread(target=run_streamlit).start()

# Expose the Streamlit app to the web using LocalTunnel
!lt --port 8501

your url is: https://giant-rules-bathe.loca.lt
